<a href="https://colab.research.google.com/github/MorgenPronk/LSTM_System_Identification/blob/main/TwoPhase_ESP_Mech_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

def esp_two_phase_head(Qin, kG, N, qL, qG, pump_data):
    """
    Calculates ESP two-phase head based on the mechanistic model.

    Args:
        Qin: Inlet flow rate (m3/s)
        kG: Gas intake volumetric fraction (-)
        N: Rotational speed (RPM)
        qL: Liquid phase density (kg/m3)
        qG: Gas phase density (kg/m3)
        pump_data: Dictionary containing pump geometry and QBM

    Returns:
        HESP: Pump head in meters of fluid (m)
    """

    # Extract pump data
    R1, R2, YI, ZI, b2, QBM = pump_data["R1"], pump_data["R2"], pump_data["YI"], pump_data["ZI"], pump_data["b2"], pump_data["QBM"]

    # Initial guess for leakage flow rate
    QLK1 = 0.05 * Qin

    # Iterate until leakage flow rate converges
    while True:
        # Calculate single-phase liquid head (refer to Fig. 7 flow chart)
        HESP_liquid = calculate_single_phase_head(Qin, N, qL, pump_data)

        # Calculate pressure increment for single-phase liquid flow
        DP = HESP_liquid * qL * 9.81

        # Calculate boosted energy term
        e = DP * Qin / (qL * pump_data["VolI"]) * (0.056 / R2)**2

        # Calculate Sauter mean diameter
        d32 = 6.034 * kG**(3/5) * qL**(1/5) * e**(2/5) / (qG**(1/5))

        # Calculate maximum bubble diameter
        dmax = d32 / 0.43

        # Calculate slip velocity and in-situ gas void fraction iteratively
        aG = kG  # Initial guess
        while True:
            # Calculate slip velocity
            VSL = math.sqrt((4 * dmax) / (3 * CD) * (qL - qG) / qL * R2 * (N * math.pi/30)**2)

            # Calculate in-situ gas void fraction
            RS = VSL * (2 * math.pi * R2 * ZI * pump_data["TB"] * YI) / (Qin + QLK1)
            aG_new = (RS - math.sqrt(RS**2 + 4 * RS * kG)) / (2 * RS)

            # Check for convergence
            if abs(aG_new - aG) < 1e-6:
                aG = aG_new
                break
            aG = aG_new

        # Determine flow pattern and drag coefficient
        acrit = 0.5 - 0.25 * math.exp(-(N / 3500)**4)
        if aG < acrit:
            # Bubble flow
            Sr = dmax * (N * math.pi/30) / VSL
            CD_0 = 24 / Re * (1 + 0.15 * Re**0.687)
            CD = CD_0 * (1 + 0.3 * Sr**2.5)
        else:
            # Intermittent flow
            # (Implementation of Eq. 73 requires additional calculations for mixture viscosity and superficial velocities)
            # ...

        # Calculate mixture densities
        qI = (1 - aG) * qL + aG * qG
        qD = (1 - kG) * qL + kG * qG

        # Calculate Euler head and pressure losses (refer to Fig. 7 flow chart)
        # (Implementation requires functions for calculating head losses)
        # ...

        # Calculate ESP boosting pressure
        PESP = qI * g * HE - PR - qI * g * HTI - qD * g * HTD - qI * g * HFI - qD * g * HFD - qD * g * HLeakage

        # Calculate pump head in meters of fluid
        HESP = PESP / (qL * 9.81)

        # Update leakage flow rate and check for convergence
        QLK2 = calculate_leakage_flow_rate(...)  # Implement leakage flow rate calculation
        if abs(QLK2 - QLK1) < 1e-3:
            break
        QLK1 = QLK2

    return HESP